In [ ]:
!pip install numpy pandas matplotlib scikit-learn tensorflow

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf

In [ ]:
# Load dataset
data = pd.read_csv('Google_Stock_Price_Train.csv')
prices = data['Open'].values.reshape(-1, 1)

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
scaled_prices = scaler.fit_transform(prices)

In [ ]:
# Prepare training data
X_train = []
y_train = []

# Use past 60 prices to predict the next one
for i in range(60, len(scaled_prices)):
    X_train.append(scaled_prices[i-60:i, 0])
    y_train.append(scaled_prices[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape input to [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [ ]:
# Build the RNN
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.SimpleRNN(50),
    tf.keras.layers.Dense(1)
])

In [ ]:
# Compile
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train
model.fit(X_train, y_train, epochs=20, batch_size=32)


In [ ]:
# Prepare test data
test_data = scaled_prices[-80:]  # Use last part of train set to simulate test
X_test = []
y_actual = scaled_prices[-20:]

In [ ]:
for i in range(60, 80):
    X_test.append(test_data[i-60:i, 0])

In [ ]:
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Predict
y_pred_scaled = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_true = scaler.inverse_transform(y_actual)

In [ ]:
# Plot results
plt.figure(figsize=(10,6))
plt.plot(y_true, label='Actual Google Stock Price')
plt.plot(y_pred, label='Predicted Google Stock Price')
plt.title('Google Stock Price Prediction using RNN')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Evaluate
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)

In [ ]:

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")